<a href="https://colab.research.google.com/github/palomadevfullstack/portfolio/blob/main/Projeto_ETL_Ecommerce_Data_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importação e Extração de Dados Fictícios
import pandas as pd
import numpy as np

# 1. Extração: vendas
data_vendas = {
    'transaction_id': [101, 102, 103, 104, 105, 106],
    'product_sku': ['SKU001', 'SKU002', 'SKU001', 'sku003', 'SKU002', 'SKU004'],
    'customer_id': ['C100', 'C101', 'C100', 'C102', 'C101', 'C103'],
    'date': ['2025-01-01', '2025-01-01', '2025-01-02', '2025-01-02', '2025-01-03', '2025-01-03'],
    'quantity': [1, 2, 1, 3, 1, 1],
    'revenue': [50.00, 150.00, 50.00, 30.00, 75.00, 200.00]
}
df_vendas = pd.DataFrame(data_vendas)

# 2. Extração: produtos
data_produtos = {
    'SKU': ['SKU001', 'SKU002', 'SKU003', 'SKU004', 'SKU005'],
    'name': ['Laptop Básico', 'Monitor 27"', 'Mouse Gamer', 'Câmera Full HD', 'Teclado'],
    'category': ['Electronics', 'Electronics', 'Peripherals', np.nan, 'Peripherals'],
    'cost': [30.00, 50.00, 5.00, 100.00, 15.00]
}
df_produtos = pd.DataFrame(data_produtos)

# 3. Extração: clientes
data_clientes = {
    'ID': ['C100', 'C101', 'C102', 'C103', 'C104'],
    'name': ['Ana Silva', 'Bruno Costa', 'Carlos Dantas', 'Daniela Reis', 'Eduarda Souza'],
    'city': ['São Paulo', 'Rio de Janeiro', 'são paulo', 'Porto Alegre', 'Curitiba']
}
df_clientes = pd.DataFrame(data_clientes)

print("--- DataFrames Extraídos ---")
print("\nDF Vendas:")
print(df_vendas.head())
print("\nDF Produtos:")
print(df_produtos.head())


--- DataFrames Extraídos ---

DF Vendas:
   transaction_id product_sku customer_id        date  quantity  revenue
0             101      SKU001        C100  2025-01-01         1     50.0
1             102      SKU002        C101  2025-01-01         2    150.0
2             103      SKU001        C100  2025-01-02         1     50.0
3             104      sku003        C102  2025-01-02         3     30.0
4             105      SKU002        C101  2025-01-03         1     75.0

DF Produtos:
      SKU            name     category   cost
0  SKU001   Laptop Básico  Electronics   30.0
1  SKU002     Monitor 27"  Electronics   50.0
2  SKU003     Mouse Gamer  Peripherals    5.0
3  SKU004  Câmera Full HD          NaN  100.0
4  SKU005         Teclado  Peripherals   15.0


In [2]:
# Transformação de Dados

# 1. Padronização produto
df_vendas['product_sku'] = df_vendas['product_sku'].str.upper()
df_produtos.rename(columns={'SKU': 'product_sku'}, inplace=True) # Renomeia para facilitar o JOIN

# 2. Padronização clientes
df_clientes.rename(columns={'ID': 'customer_id'}, inplace=True)

# Limpeza de Dados nulos
df_produtos['category'].fillna('Desconhecida', inplace=True)

# Padronização cidade dos clientes
df_clientes['city'] = df_clientes['city'].str.title()

print("--- DataFrames Transformados (Limpeza) ---")
print("\nDF Vendas (Chave Padronizada):")
print(df_vendas)
print("\nDF Clientes (Cidades Padronizadas):")
print(df_clientes)


--- DataFrames Transformados (Limpeza) ---

DF Vendas (Chave Padronizada):
   transaction_id product_sku customer_id        date  quantity  revenue
0             101      SKU001        C100  2025-01-01         1     50.0
1             102      SKU002        C101  2025-01-01         2    150.0
2             103      SKU001        C100  2025-01-02         1     50.0
3             104      SKU003        C102  2025-01-02         3     30.0
4             105      SKU002        C101  2025-01-03         1     75.0
5             106      SKU004        C103  2025-01-03         1    200.0

DF Clientes (Cidades Padronizadas):
  customer_id           name            city
0        C100      Ana Silva       São Paulo
1        C101    Bruno Costa  Rio De Janeiro
2        C102  Carlos Dantas       São Paulo
3        C103   Daniela Reis    Porto Alegre
4        C104  Eduarda Souza        Curitiba


/tmp/ipython-input-1924878229.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_produtos['category'].fillna('Desconhecida', inplace=True)


In [3]:
# Tranformação / Integração
#Juntar produtos com Vendas
df_stage = pd.merge(
    df_vendas,
    df_produtos,
    on='product_sku',
    how='left'
)

# Juntar o resultado com Clientes
df_datamart = pd.merge(
    df_stage,
    df_clientes,
    on='customer_id',
    how='left'
)

df_datamart['total_cost'] = df_datamart['cost'] * df_datamart['quantity']
df_datamart['profit'] = df_datamart['revenue'] - df_datamart['total_cost']

print("--- Data Mart Final (Primeiras 5 linhas) ---")
print(df_datamart.head())

# Inspecção final
print("\nDF Info Final:")
print(df_datamart.info())


--- Data Mart Final (Primeiras 5 linhas) ---
   transaction_id product_sku customer_id        date  quantity  revenue  \
0             101      SKU001        C100  2025-01-01         1     50.0   
1             102      SKU002        C101  2025-01-01         2    150.0   
2             103      SKU001        C100  2025-01-02         1     50.0   
3             104      SKU003        C102  2025-01-02         3     30.0   
4             105      SKU002        C101  2025-01-03         1     75.0   

          name_x     category  cost         name_y            city  \
0  Laptop Básico  Electronics  30.0      Ana Silva       São Paulo   
1    Monitor 27"  Electronics  50.0    Bruno Costa  Rio De Janeiro   
2  Laptop Básico  Electronics  30.0      Ana Silva       São Paulo   
3    Mouse Gamer  Peripherals   5.0  Carlos Dantas       São Paulo   
4    Monitor 27"  Electronics  50.0    Bruno Costa  Rio De Janeiro   

   total_cost  profit  
0        30.0    20.0  
1       100.0    50.0  
2    

In [4]:
# Carga e Simulação

colunas_finais = [
    'date', 'transaction_id', 'customer_id', 'product_sku', 'name_x', 'category',
    'quantity', 'revenue', 'cost', 'total_cost', 'profit', 'city'
]
df_final_load = df_datamart[colunas_finais].copy()

# Salvamento (Simulação do 'L' de Load)
df_final_load.to_parquet('ecommerce_data_mart.parquet', index=False)

print("\n[SUCESSO] Pipeline ETL concluído!")
print("O Data Mart foi salvo como 'ecommerce_data_mart.parquet'.")
print("Pronto para ser consumido por analistas ou modelos de ML.")


[SUCESSO] Pipeline ETL concluído!
O Data Mart foi salvo como 'ecommerce_data_mart.parquet'.
Pronto para ser consumido por analistas ou modelos de ML.


# 5. Conclusão de Engenharia de Dados: Pipeline ETL

## Demonstração de Habilidades
Este projeto demonstrou as habilidades essenciais de Engenharia de Dados:
* **Extração (E):** Coleta de dados de múltiplas fontes (simuladas em DataFrames).
* **Transformação (T):** Tratamento de qualidade (padronização de `SKU`, `city`), tratamento de nulos (`category`) e **integração** (`pd.merge`).
* **Modelagem/Feature Engineering:** Criação de novas colunas de negócio (`total_cost`, `profit`).
* **Carga (L):** Persistência do resultado em um formato otimizado (`.parquet`), pronto para consumo.

## Esquema de Dados (Data Mart)

| Coluna | Tipo | Origem | Finalidade |
| :--- | :--- | :--- | :--- |
| `date` | datetime | Vendas | Chave Temporal para Análise de Séries |
| `product_sku` | string | Vendas/Produtos | Chave de Ligação |
| `category` | string | Produtos | Variável de Agrupamento |
| `revenue` | float | Vendas | Variável de Negócio |
| `profit` | float | Transf. | KPI Principal (Lucro da Transação) |
| `city` | string | Clientes | Variável Geográfica |

Este Data Mart unificado está em um formato que reduz o tempo de consulta para Analistas de Dados, cumprindo a função principal de um Engenheiro de Dados.